# Quantum Credit Risk Analysis

*Authors: Arjun Puppala, Jose Ramon Aleman, Marti Ciurana*
*Tutor: Giulio Gasbarri*

## Abstract

...


## Introduction

Credit risk analysis is a form of analysis performed by a credit analyst to determine a borrower’s ability to meet their debt obligations. The purpose of credit analysis is to determine the creditworthiness of borrowers by quantifying the risk of loss that the lender is exposed to. The three factors that lenders use to quantify credit risk include the probability of default, loss given default, and exposure at default. [1]

The use of quantum computing could allow faster and accurate solutions, for example determining a credit risk profile of a client. It is said that financial institutions that can make the most of quantum computing are likely to see significant benefits. Banks and asset managers optimize portfolios based on computationally intense models that process large sets of variables. For this reason, they will be able to more effectively analyze large or unstructured data sets.

In valuation, for example, the ability to speedily identify an optimal risk-adjusted portfolio is likely to create significant competitive advantage. For loan and bond portfolios, more precise estimates of credit exposures should lead to better optimization decisions.

American and European banks are also exploring quantum computing opportunities. [2]

- Bank of America strategist said quantum computing would be “as revolutionary in the 2020s as smartphones were in the 2010s.”
- BBVA has formed a partnership to explore portfolio optimization and more efficient Monte Carlo modelling.
- Caixa Bank is running a trial hybrid framework of quantum and conventional computing with the aim of better classifying credit risk profiles.

## Description of the Problem

Default risk is the risk that a lender takes on in the chance that a borrower will be unable to make the required payments on their debt obligation. 

Whenever a lender extends credit to a borrower, there is a chance that the loan amount will not be paid back. The measurement that looks at this probability is the default risk. Default risk does not only apply to individuals who borrow money, but also to companies that issue bonds and due to financial constraints, are not able to make interest payments on those bonds. Whenever a lender extends credit, calculating the default risk of a borrower is crucial as part of its risk management strategy. Whenever an investor is evaluating an investment, determining the financial health of a company or private person is crucial in gauging investment risk.

Lenders generally examine a company's financial statements and employ several financial ratios to determine the likelihood of debt repayment. Free cash flow is the cash that is generated after the company reinvests in itself and is calculated by subtracting capital expenditures from operating cash flow. Free cash flow is used for things such as debt and dividend payments. A free cash flow figure that is near zero or negative indicates that the company may be having trouble generating the cash necessary to deliver on promised payments. This could indicate a higher default risk.

The risk of default on a debt arises from a borrower failing to make the required payments, for example:

- A consumer may fail to make a payment due on a mortgage loan, credit card, line of credit, or other loan.
- A company is unable to repay asset-secured fixed or floating charge debt.
- A business or consumer does not pay a trade invoice when due

## Methods

Value at risk (VaR) is a statistic that quantifies the extent of possible financial losses within a firm, portfolio, or position over a specific time frame. 

A financial firm, for example, may determine an asset has a 3% one-month VaR of 2%, representing a 3% chance of the asset declining in value by 2% during the one-month time frame. The conversion of the 3% chance of occurrence to a daily ratio places the odds of a 2% loss at one day per month.

The CVar quantifies the average expected loss.

### Montecarlo

The Monte Carlo method is a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. The underlying concept is to use randomness to solve problems that might be deterministic in principle. Monte Carlo Simulation predicts a set of outcomes based on an estimated range of values versus a set of fixed input values. 

In other words, a Monte Carlo Simulation builds a model of possible results by leveraging a probability distribution, such as a uniform or normal distribution, for any variable that has inherent uncertainty. It, then, recalculates the results over and over, each time using a different set of random numbers between the minimum and maximum values. In a typical Monte Carlo experiment, this exercise can be repeated thousands of times to produce a large number of likely outcomes.

Several combinations of different scenarios are tested and the result of each scenario is analyzed. Therefore it’s slow and expensive.

### Grover's Algorithm

In this section, we introduce Grover's algorithm and how it can be used to solve unstructured search problems. This algorithm can speed up an unstructured search problem quadratically.

[diagram goes here]

## Results

...

## References

1. [Purpose of Credit Risk Analysis](https://corporatefinanceinstitute.com/resources/knowledge/credit/purpose-of-credit-risk-analysis/)
2. [How Quantum Computing Could Change Financial Services](https://www.mckinsey.com/industries/financial-services/our-insights/how-quantum-computing-could-change-financial-services)